In [3]:
# !pip install torch

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open("rocities.txt", "r").read().splitlines()
words[:10]

['1 DECEMBRIE',
 '2 MAI',
 '23 AUGUST',
 'ABRAM',
 'ABRAMUT',
 'ABRUD',
 'ABRUD-SAT',
 'ABUCEA',
 'ABUD',
 'ABUS']

In [4]:
len(words)

10148

In [8]:
# build the vocabulary and mappings to/from ints
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i,s in enumerate(chars)}
stoi['*'] = 0
itos = {i: s for s,i in stoi.items()}
print(stoi)
chars_count = len(stoi.values())

{' ': 1, '-': 2, '.': 3, '1': 4, '2': 5, '3': 6, 'A': 7, 'B': 8, 'C': 9, 'D': 10, 'E': 11, 'F': 12, 'G': 13, 'H': 14, 'I': 15, 'J': 16, 'K': 17, 'L': 18, 'M': 19, 'N': 20, 'O': 21, 'P': 22, 'R': 23, 'S': 24, 'T': 25, 'U': 26, 'V': 27, 'X': 28, 'Y': 29, 'Z': 30, '*': 0}


In [9]:
# dataset

block_size = 3 # length of context: no. of characters we use to predict the next

X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '*':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '-->', itos[ix])
        context = context[1:] + [ix] # crop and append, like a rolling window
        
X = torch.tensor(X)
Y = torch.tensor(Y)

1 DECEMBRIE
*** --> 1
**1 -->  
*1  --> D
1 D --> E
 DE --> C
DEC --> E
ECE --> M
CEM --> B
EMB --> R
MBR --> I
BRI --> E
RIE --> *
2 MAI
*** --> 2
**2 -->  
*2  --> M
2 M --> A
 MA --> I
MAI --> *
23 AUGUST
*** --> 2
**2 --> 3
*23 -->  
23  --> A
3 A --> U
 AU --> G
AUG --> U
UGU --> S
GUS --> T
UST --> *
ABRAM
*** --> A
**A --> B
*AB --> R
ABR --> A
BRA --> M
RAM --> *
ABRAMUT
*** --> A
**A --> B
*AB --> R
ABR --> A
BRA --> M
RAM --> U
AMU --> T
MUT --> *


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([42, 3]), torch.int64, torch.Size([42]), torch.int64)

In [10]:
chars_count

31

In [11]:
C = torch.randn((chars_count, 2))  # lookup table - each char is 2 dimensional

In [13]:
emb = C[X]
emb.shape

torch.Size([42, 3, 2])

In [14]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [15]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [16]:
h

tensor([[ 0.9534,  0.9526, -0.9999,  ...,  0.3725,  0.8822,  0.9132],
        [ 0.9981,  0.9784, -1.0000,  ...,  0.8397,  0.9609,  0.9558],
        [-0.9837,  0.9834, -0.9677,  ...,  0.9980, -0.6042,  0.5047],
        ...,
        [ 0.9984, -0.1281, -0.9944,  ..., -0.9721, -0.4807,  0.0122],
        [ 0.8547,  0.7818, -0.9986,  ...,  0.9968, -0.9415,  0.9750],
        [-0.8325,  1.0000, -0.9164,  ...,  0.9927,  0.6431,  0.6123]])

In [18]:
h.shape

torch.Size([42, 100])

In [27]:
W2 = torch.randn((100, chars_count))
b2 = torch.randn(chars_count)

In [28]:
logits = h @ W2 + b2

In [29]:
logits.shape

torch.Size([42, 31])

In [24]:
counts = logits.exp()

In [25]:
prob = counts / counts.sum(1, keepdims=True)

In [26]:
prob.shape

torch.Size([42, 27])

In [30]:
loss = -prob[torch.arange(42), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [42]